## Basic Search Demo

Demonstrates the basic use of the Best-First Search idea on the 8-puzzle.

In [2]:
# imports and setup
import random
import numpy as np
from enum import Enum
import heapq

from IPython.display import Image

### Defining some basic elements of the 8-puzzle domain
We begin by specifying some of the basic elements of the 8-puzzle, including the 4 actions available, and the solution goal-state. States will be encoded as 2-dimensional integer arrays, where 0 is used to represent the blank square.

In [ ]:
# Grid constant
GRID_SIZE = 3

# Action constants
class Action(Enum):
    UP = 1
    RIGHT = 2
    LEFT = 3
    DOWN = 4
    
# Goal-state constant
GOAL_STATE = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

In [ ]:
def action_is_legal(puzzle_state, action):
    '''Determines whether a given action is legal for a puzzle-state.
       Actions are legal if and only if taking that action would not
       require moving the blank square outside the grid.
       
       Args
       ----
       puzzle_state: a 2-dimensional array containing some permutation of]
           the integers in [0,8]
       action: some one of the actions available in the enumeration Action
       
       Returns
       -------
       boolean value: true if and only if action is legal in puzzle_state
    '''
    is_blank = np.argwhere(puzzle_state==0)
    blank_row = is_blank[0][0]
    blank_col = is_blank[0][1]
    
    return (action == Action.UP and blank_row > 0) \
        or (action == Action.DOWN and blank_row < GRID_SIZE - 1) \
        or (action == Action.LEFT and blank_col > 0) \
        or (action == Action.RIGHT and blank_col < GRID_SIZE - 1)

In [ ]:
def get_next_state(puzzle_state, action):
    '''Returns next state reached from a state, after an action.
       
       Args
       ----
       puzzle_state: a 2-dimensional array containing some permutation of]
           the integers in [0,8]
       action: some one of the actions available in the enumeration Action
       
       Returns
       -------
       boolean value: the next state after taking action in puzzle_state;
           represented as 2-dimensional array, like puzzle_state
           
       Exceptions
       ----------
       Exception raised if not action_is_legal(puzzle_state, action)
    '''
    next_state = np.ndarray.copy(puzzle_state)
    
    is_blank = np.argwhere(puzzle_state==0)
    blank_row = is_blank[0][0]
    blank_col = is_blank[0][1]
    
    if not action_is_legal(puzzle_state, action):
        raise Exception(
            'Action {} not legal; position of blank is [{}, {}]'.format(action.name, blank_row, blank_col))
    
    target_row = blank_row
    target_col = blank_col
    
    if action == Action.UP:
        target_row = target_row - 1
    elif action == Action.DOWN:
        target_row = target_row + 1
    elif action == Action.LEFT:
        target_col = target_col - 1
    elif action == Action.RIGHT:
        target_col = target_col + 1
                
    next_state[blank_row][blank_col], next_state[target_row][target_col] = \
        next_state[target_row][target_col], next_state[blank_row][blank_col]
    
    return next_state

In [ ]:
def get_possible_actions(puzzle_state):
    '''Generates lists of possible actions and the states that result from them,
       starting in puzzle_state.
       
       Args
       ----
       puzzle_state: a 2-dimensional array containing some permutation of]
           the integers in [0,8]
       
       Returns
       -------
       action_list: list of Action items legal in puzzle_state
       next_state_list: list of puzzle configurations reachable by taking actions
           Note: each element of next_state_list is the 2-dimensional array
                 configuration that is reached after taking the matching action
                 from action_list in puzzle_state
    '''
    action_list = list()
    next_state_list = list()
    
    is_blank = np.argwhere(puzzle_state==0)
    blank_row = is_blank[0][0]
    blank_col = is_blank[0][1]
    
    for action in Action:
        if action_is_legal(puzzle_state, action):
            action_list.append(action)
            next_state_list.append(get_next_state(puzzle_state, action))
    
    return action_list, next_state_list

### Generating a random starting state
In order to initialize an 8-puzzle instance properly, we need to ensure that the instance is actually solvable.  Simply generating some random arrangement of the "tiles" (0..8) will *not* work in general–if we do so, it is possible that there is no actual way to use legal moves to get to the desired solution.

One way of guaranteeing solvable instances is to begin with the desired goal state, and then make random moves from that configuration.  This guarantees that we can, at worst, reverse that sequence of moves to solve the puzzle; of course, it may be that there is some shorter solution, since our random moves may, for example, repeat configurations along the way.

### Encoding a data structure for search-tree nodes

In [ ]:
Image(url="search_tree_node.pdf", width=500)

### Implementing basic best-first search

In [4]:
Image(url="bestFirstSearch.pdf", width=600)